# SD212: Graph mining
## Lab 6: Spectral embedding

In this lab, you will learn to embed the nodes of a graph in a vector space of low dimension. We consider the  embedding based on the top eigenvectors of the transition matrix $P=D^{-1}A$.

## Import

In [ ]:
from IPython.display import SVG

In [ ]:
import numpy as np
from scipy import sparse
from matplotlib import pyplot as plt

In [ ]:
from sknetwork.data import load_netset, karate_club
from sknetwork.embedding import Spectral
from sknetwork.ranking import PageRank
from sknetwork.visualization import svg_graph

## Data

We will work on the following graphs (see the [NetSet](https://netset.telecom-paris.fr/) collection for details):
* Openflights (graph)
* WikiVitals (directed graph and bipartite graph)

In [ ]:
openflights = load_netset('openflights')
wikivitals = load_netset('wikivitals')

## 1. Graphs

## Karate Club


We first consider the spectral embedding of the [karate club graph](https://en.wikipedia.org/wiki/Zachary%27s_karate_club).

In [ ]:
dataset = karate_club(metadata=True)

In [ ]:
adjacency = dataset.adjacency
position = dataset.position
labels_true = dataset.labels

In [ ]:
image = svg_graph(adjacency, position, labels=labels_true)
SVG(image)

## To do

* Display the spectrum of the transition matrix (e.g., first 20 eigenvalues). 
* What does the spectrum suggest?
* Display the graph with some eigenvectors.
* Display the embedding of the graph in dimension 2.
* Compare the clusters obtained with the sign of the first dimension to the ground-truth clusters.

In [ ]:
spectral = Spectral(20, normalized=False)

In [ ]:
spectral.fit(adjacency)

In [ ]:
# add first eigenvalue
eigenvalues = [1] + list(spectral.eigenvalues_)

In [ ]:
plt.scatter(np.arange(len(eigenvalues)) + 1, eigenvalues, color='r', lw=3)
plt.ylim(-1.1,1.1)

The spectrum suggests the presence of 4 clusters (as there are 4 dominant eigenvalues). Interestingly, this is the number of clusters returned by Louvain.

In [ ]:
eigenvectors = spectral.eigenvectors_

In [ ]:
# display top eigenvector
image = svg_graph(adjacency, position, scores=eigenvectors[:, 0])
SVG(image)

In [ ]:
# next eigenvector
image = svg_graph(adjacency, position, scores=eigenvectors[:, 1])
SVG(image)

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(eigenvectors[:, 0], eigenvectors[:, 1], s=100);

In [ ]:
colors = ['r', 'b']
plt.figure(figsize=(5,5))
for label in np.unique(labels_true):
    plt.scatter(eigenvectors[labels_true==label, 0], eigenvectors[labels_true==label, 1], s=100, c=colors[label]);
plt.axvline(c='k');

In [ ]:
labels_pred = (eigenvectors[:,0] > 0).astype(int)

In [ ]:
image = svg_graph(adjacency, position, labels=labels_pred)
SVG(image)

In [ ]:
np.mean(labels_pred==labels_true)

## Openflights


We now consider a larger graph. We use spectral embedding in dimension 20 to cluster the graph by k-means in the embedding space.

In [ ]:
dataset = openflights

In [ ]:
adjacency = dataset.adjacency
position = dataset.position
names = dataset.names

In [ ]:
image = svg_graph(adjacency, position, width=800, height=400, node_size=3, display_edges=False)
SVG(image)

## To do

* Display the same world map with 8 clusters found by k-means in the embedding space.
* Do the same without normalization on the unit sphere (``normalized=False``).<br> Interpret the results. You might compute the distance of the barycenter of each cluster to the origin.

In [ ]:
k = 20
spectral = Spectral(k, normalized=True)

In [ ]:
embedding = spectral.fit_transform(adjacency)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(8, n_init=10)

In [ ]:
labels = kmeans.fit_predict(embedding)

In [ ]:
image = svg_graph(adjacency, position, width=800, height=400, node_size=3, labels=labels, display_edges=False)
SVG(image)

In [ ]:
spectral = Spectral(k, normalized=False)

In [ ]:
embedding = spectral.fit_transform(adjacency)

In [ ]:
kmeans = KMeans(8, n_init=10)

In [ ]:
labels = kmeans.fit_predict(embedding)

In [ ]:
image = svg_graph(adjacency, position, width=800, height=400, node_size=3, labels=labels, display_edges=False)
SVG(image)

In [ ]:
for label in np.unique(labels):
    mask = labels==label
    print(label, np.sum(mask), np.linalg.norm(np.mean(embedding[mask], axis=0)))

In the absence of normalization, the small subgraphs that are weakly connected to the rest of the graph are isolated by the spectral embedding. Projecting the embedding on the unit sphere allows to see the main structure of the graph.

## 2. Directed and bipartite graphs

We now work on directed graph and bipartite graphs. We measure proximity between nodes in terms of [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). Equivalently, we consider the embedding on the unit sphere (``normalized=True``).

## Wikipedia Vitals

In [ ]:
dataset = wikivitals

In [ ]:
adjacency = dataset.adjacency
biadjacency = dataset.biadjacency
names = dataset.names
words = dataset.names_col
labels = dataset.labels
names_labels = dataset.names_labels
labels_hierarchy = dataset.labels_hierarchy
names_labels_hierarchy = dataset.names_labels_hierarchy

## To do

We first consider the graph of links.

* List the 10 articles that are closest to **Vincent van Gogh** in terms of cosine similarity in the embedding space.
* Display the 3D-plot of each label in the embedding space (top 3 dimensions). <br>You might represent each label by a point located at the barycenter of the corresponding articles, with a size proportional to the number of articles.
* Display the dendrogram of the top-100 articles on **Arts** (in terms of Personalized PageRank) given by the [Ward method](https://en.wikipedia.org/wiki/Ward%27s_method) in the embedding space (hierarchical clustering).

In [ ]:
spectral = Spectral(20)

In [ ]:
embedding = spectral.fit_transform(adjacency)

In [ ]:
node = int(np.flatnonzero(names=='Vincent van Gogh'))

In [ ]:
# cosine similarity
scores = embedding.dot(embedding[node])

In [ ]:
print(names[np.argsort(-scores)[:10]])

In [ ]:
labels_unique, counts = np.unique(labels, return_counts=True)

In [ ]:
embedding_label = np.array([np.mean(embedding[labels==label], axis=0) for label in labels_unique])

In [ ]:
embedding_label.shape

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
dataframe = pd.DataFrame(embedding_label[:, :3], columns=list('xyz'))

In [ ]:
dataframe['category'] = names_labels[labels_unique]
dataframe['count'] = counts

In [ ]:
fig = px.scatter_3d(dataframe, x='x', y='y', z='z', text='category', color='category', size='count', size_max=100, opacity=0.5)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
print(names_labels)

In [ ]:
pagerank = PageRank()

In [ ]:
scores = pagerank.fit_predict(adjacency, labels==0)

In [ ]:
scores *= labels==0

In [ ]:
top = np.argsort(-scores)[:100]

In [ ]:
from scipy.cluster.hierarchy import linkage

In [ ]:
dendrogram = linkage(embedding[top], method='ward')

In [ ]:
from sknetwork.visualization import svg_dendrogram

In [ ]:
SVG(svg_dendrogram(dendrogram, names=names[top], rotate=True, rotate_names=True, height=1000))

## To do

* Repeat the same experiments of the bipartite graph between articles and words.
* List the 10 articles and the 10 words that are the closest to the word **painting** in the embedding space.

In [ ]:
embedding = spectral.fit_transform(biadjacency)

In [ ]:
embedding_words = spectral.embedding_col_

In [ ]:
# cosine similarity with articles
scores = embedding.dot(embedding_words[words=='painting'].ravel())

In [ ]:
print(names[np.argsort(-scores)[:10]])

In [ ]:
# cosine similarity with words
scores = embedding_words.dot(embedding_words[words=='painting'].ravel())

In [ ]:
print(words[np.argsort(-scores)[:10]])

## To do

* Check that the average cosine similarity between nodes in some set $S$ is given by the square distance of the barycenter of $S$ to the origin.

$$
\frac 1 {n^2}\sum_{i, j \in S}x_i.x_j = \left(\frac 1 n \sum_{i\in S}x_i\right).\left(\frac 1 n \sum_{j\in S}x_j\right) = ||\frac 1 n \sum_{i\in S}x_i||^2
$$

* Deduce the average cosine similarity between articles of the **Mammals** category.
* Compare with the expected cosine similarity between two articles sampled uniformly at random.
* Defining a category as **topical** if its average cosine similarity is close to 1, rank the 11 categories (Arts, History,...) by topicality. 
* List the 10 most topical and the 10 less topical hierarchical categories having at least 10 articles (like **Mammals**). 

In [ ]:
biadjacency = dataset.biadjacency

In [ ]:
spectral = Spectral(20)

In [ ]:
embedding = spectral.fit_transform(biadjacency)

In [ ]:
{label: name for label, name in enumerate(names_labels_hierarchy) if name.endswith('Mammals')}

In [ ]:
mammal = 329

In [ ]:
def get_average_cosine(mask):
    return np.linalg.norm(np.mean(embedding[mask], axis=0))**2

In [ ]:
get_average_cosine(labels_hierarchy==mammal)

In [ ]:
get_average_cosine(labels_hierarchy>=0)

In [ ]:
scores = np.array([get_average_cosine(labels==label) for label in np.unique(labels)])

In [ ]:
for label in np.argsort(-scores):
    print(scores[label], names_labels[label])

In [ ]:
labels_unique, counts = np.unique(labels_hierarchy, return_counts=True)

In [ ]:
labels_top = labels_unique[counts >= 10]
counts_top = counts[counts >=10]

In [ ]:
scores = np.array([get_average_cosine(labels_hierarchy==label) for label in labels_top])

In [ ]:
for i in np.argsort(-scores)[:10]:
    label = labels_top[i]
    count = counts_top[i]
    print(scores[i], count, names_labels_hierarchy[label])

In [ ]:
for i in np.argsort(scores)[:10]:
    label = labels_top[i]
    count = counts_top[i]
    print(scores[i], count, names_labels_hierarchy[label])